# Multi-level and Marginal Models - Linear and Logistic Regression - NHANES Dataset

This notebook applies multi-level and marginal models for linear and logistic regression to the NHANES datset. In contrast to ordinary linear and logistic regression, these models make the assumption that the variables are somehow correlated; that can be due to:

- Correlations within clusters; that happens when a clustered complex sample is done by dividing in neighborhoods, etc.
- Repeated measures along the time, typical in longitudinal studies that measure variable evolution during time.

Marginal and multi-level models differentiate from each other in these aspects:

- Multi-level models use random coefficients, i.e., variances between clusters (subject, neighborhood, etc.) are captured as random variables or effects; in addition to them, we also have the regular fixed effects or constant coefficients. All in all, the variance of the random effects is estimated. Thus, as a result, each cluster has its own model.
- Marginal models do not have random coefficients; instead, they have the regular linear/lohgistic regression parameters or coefficients. However, the variance between the variables is also computed to adjust the standard error of the coefficients. This allows for a more realistic inference (i.e., hypothesis testing or confidence interval computation). Additionally, that variance is computed overall, i.e., between-cluster variations are not computed, as it is done by multi-level models.

Overview of contents:
1. Data Analysis and Pre-Processing
    - 1.1 Cluster Ids
    - 1.2 Interclass Correlation (ICC)
    - 1.3 Conditional Interclass Correlation
2. Marginal Linear Models with Dependent Data
    - 2.1 Marginal Model for Linear Regression and Comparison with OLS
    - 2.2 Marginal Model for Logistic Regression and Comparison with GLM
3. Multi-Level Models with Dependent Data (Linear Regression)
    - 3.1 Random Effects in the Intercept
        - 3.1.1 Predicted Random Effects
    - 3.2 Random Slopes
    - 3.3 Intercepts and Slopes Allowed to Be Correlated
4. Week 3 Assessment

## 1. Data Analysis and Pre-Processing

In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [48]:
# Read the data file
da = pd.read_csv("nhanes_2015_2016.csv")

In [49]:
# Drop unused columns, keep variables for our model
# Variables that account for clustering in the complex sample
# [SDMVSTRA](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVSTRA)
# [SDMVPSU](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVPSU)
vars = ["BPXSY1",
        "RIDAGEYR",
        "RIAGENDR",
        "RIDRETH1",
        "DMDEDUC2",
        "BMXBMI",
        "SMQ020",
        "SDMVSTRA",
        "SDMVPSU"]
# Drop rows with any missing values
da = da[vars].dropna()

### 1.1 Cluster Ids

Roughly speaking, in NHANES the data
are collected by selecting a limited number of counties in the US,
then selecting subregions of these counties, then selecting people
within these subregions.  Since counties are geographically
constrained, it is expected that people within a county are more
similar to each other than they are to people in other counties.

For privacy reasons, county ids are encoded as "masked variance units" (MVUs), which can be retrieved with `SDMVSTRA` and `SDMVPSU`.

In [50]:
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

### 1.2 Interclass Correlation (ICC)

Interclass Correlation (ICC) can be used to measure the dependence of the variables within clusters or groups. `ICC = 1` means perfect dependence. Note that ICC is similar but not the same as the Pearson's correlation; small values like `0.03` are not that small in ICC.

In [88]:
# ICC of blood pressure
# Small values like `0.03` are not that small in ICC.
# Formula "BPXSY1 ~ 1" means only intercept for blood pressure
model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


In [52]:
# Recode smoking to a simple binary variable
da["smq"] = da.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

In [89]:
# ICC of all variables
# Similar values to the blood pressure
# except for SDMVSTRA - as expected, since it is part of the grop ID
for v in ["BPXSY1", "RIDAGEYR", "BMXBMI", "smq", "SDMVSTRA"]:
    model = sm.GEE.from_formula(v + " ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
RIDAGEYR The correlation between two observations in the same cluster is 0.035
BMXBMI The correlation between two observations in the same cluster is 0.039
smq The correlation between two observations in the same cluster is 0.026
SDMVSTRA The correlation between two observations in the same cluster is 0.959


### 1.3 Conditional Interclass Correlation

In [54]:
# The ICC model above is computed without taking into account other variables
# Here we control the age variable
# We know that at older ages the blood pressure increases; that is an explained variation
# Thus, we update the model to compute more reliable ICCs
# and indeed the ICC value decreases
# Lesson: always introduce control variables in the model before computing the ICC
model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.019


In [55]:
# Create a labeled version of the gender variable
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [56]:
# Again, we refine the model and introduce other variables that might explain the outcome
# That way, the ICC is more accurate
# C(RIDRETH1): 5 levels of ethnicity; C() convertes integers to categorical levels
model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.013


## 2. Marginal Models with Dependent Data

### 2.1 Marginal Model for Linear Regression and Comparison with OLS

If we have dependent data and we use ordinary linear models (i.e., not multi-level or marginal models), the mean parameters or coefficients will be correct, but the standard errors won't be correct. Thus, the significances of the parameters will be wrong. Therefore, GEE should be used to fit dependent data with continuous outcomes instead of OLS.

In [57]:
# Fit a linear model with OLS
model1 = sm.OLS.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           data=da)
result1 = model1.fit()

In [58]:
# Fit a marginal linear model using GEE to handle dependent data
model2 = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result2 = model2.fit()

In [59]:
# Create dataframe which contains model parameters and standard errors
# Linear regression: OLS vs GEE (assuming dependent data)
x = pd.DataFrame({"OLS_params": result1.params, "OLS_SE": result1.bse,
                  "GEE_params": result2.params, "GEE_SE": result2.bse})
x = x[["OLS_params", "OLS_SE", "GEE_params", "GEE_SE"]]
print(x)

                   OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept           91.736583  1.339378   92.168530  1.384309
RIAGENDRx[T.Male]    3.671294  0.453763    3.650245  0.454498
C(RIDRETH1)[T.2]     0.855488  0.819486    0.159296  0.767025
C(RIDRETH1)[T.3]    -1.796132  0.671954   -2.233280  0.760228
C(RIDRETH1)[T.4]     3.813314  0.732355    3.105654  0.881580
C(RIDRETH1)[T.5]    -0.455347  0.808948   -0.439831  0.813675
RIDAGEYR             0.478699  0.012901    0.474101  0.018493
BMXBMI               0.278015  0.033285    0.280205  0.038553


### 2.2 Marginal Model for Logistic Regression and Comparison with GLM

Similarly as before, GEE should be used to fit dependent data with binary outcomes instead of GLM.

In [60]:
# Relabel the levels, convert rare categories to missing.
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

In [61]:
# Fit a basic GLM
model1 = sm.GLM.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           family=sm.families.Binomial(), data=da)
result1 = model1.fit()
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    smq   No. Observations:                 5093
Model:                            GLM   Df Residuals:                     5086
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3201.2
Date:                Thu, 31 Mar 2022   Deviance:                       6402.4
Time:                        09:05:51   Pearson chi2:                 5.10e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -2.3060      0.114    -20.174      0.000      -2.530      -2.082
RIAGENDRx[T.Male]               0.9096      0.060     15.118      0.000       0.792       1.028
C(DMDEDUC2x)[T.HS]              0.9434      0.090     10.521      0.000       0.768       1.119
C(DMDEDUC2x)[T.SomeCollege]     0.8322      0.084      9.865      0.000       0.667       0.998
C(DMDEDUC2x)[T.lt9]             0.2662      0.109      2.438      0.015       0.052       0.480
C(DMDEDUC2x)[T.x9_11]           1.0986      0.107     10.296      0.000       0.889       1.308
RIDAGEYR                        0.0183      0.002     10.582      0.000       0.015       0.022
===============================================================================================
"""

In [62]:
# Fit a marginal GLM using GEE
model2 = sm.GEE.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           groups="group", family=sm.families.Binomial(),
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result2 = model2.fit(start_params=result1.params)

In [63]:
# Create dataframe which contains model parameters and standard errors
# Logistic regression: GLM vs GEE (assuming dependent data)
x = pd.DataFrame({"GLM_params": result1.params, "GLM_SE": result1.bse,
                  "GEE_params": result2.params, "GEE_SE": result2.bse})
x = x[["GLM_params", "GLM_SE", "GEE_params", "GEE_SE"]]
print(x)

                             GLM_params    GLM_SE  GEE_params    GEE_SE
Intercept                     -2.305999  0.114308   -2.249820  0.140567
RIAGENDRx[T.Male]              0.909597  0.060167    0.908682  0.062342
C(DMDEDUC2x)[T.HS]             0.943364  0.089663    0.887965  0.095397
C(DMDEDUC2x)[T.SomeCollege]    0.832227  0.084361    0.771636  0.104449
C(DMDEDUC2x)[T.lt9]            0.266228  0.109183    0.321784  0.141327
C(DMDEDUC2x)[T.x9_11]          1.098561  0.106697    1.062149  0.138401
RIDAGEYR                       0.018257  0.001725    0.017416  0.001803


## 3. Multi-Level Models with Dependent Data (Linear Regression)

Multi-level models are vast and complex topic; the course barely introduces it, without going much in detail.

Recall that the model is split in 2 levels. For example, let's consider a longitudinal study performed across the time ($t$) and in different clusters ($i$). The equations with random intercepts and slopes can be defined as:

**Level 1**: (usual fomula, but these are not constant coefficients anymore)

$y_{t,i} = \beta_{0,i} + \beta_{1,i} x_{1,t,i} + e_{t,i}$

**Level 2**: (values plugged into equation of level 1)

$\beta_{0,i} = \beta_{0,0} + \beta_{0,1} T_{i} + u_{0,i}$

$\beta_{1,i} = \beta_{1,0} + \beta_{1,1} T_{i} + u_{1,i}$

Notes:
- $y_{t,i}$: outcome of cluster $i$ at time point $t$
- $e_{t,i}$: error associated to $y_{t,i}$
- $\beta_{0,i}$, $\beta_{1,i}$: these are not constant coefficients anymore, but random coefficients, as defined in level 2
- $x_{1,t,i}$: (independent) viariable 1, time $t$, cluster $i$
- $u$: Gaussian centered around 0 but with specific variance, which is an estimated parameter. I understand that the random effect assigned to the intercepts and the slopes comes from here.
- $\beta_{0,0}$, $\beta_{0,1}$: mean intercepts of the random effects.
- $\beta_{1,0}$, $\beta_{1,1}$: mean slopes of the random effects.
- $T_i$: time variable.



### 3.1 Random Effects in the Intercept

In [81]:
# Fit a multilevel (mixed effects) model to handle dependent data
# By default, only the intercept of the random effects is included in the level 2 formula
# but here I explicitly define it: re_formula = "1"
# I understand that would be equivalent to: 
# - beta_0i = beta_0 + u_0i; 1 refers to beta_0
# - beta_1i = beta_1 + u_1i; 1 refers to beta_1
model = sm.MixedLM.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           re_formula = "1",
           data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
============================================================
Model:             MixedLM  Dependent Variable:  BPXSY1     
No. Observations:  5102     Method:              REML       
No. Groups:        30       Scale:               256.6952   
Min. group size:   106      Log-Likelihood:      -21409.8702
Max. group size:   226      Converged:           Yes        
Mean group size:   170.1                                    
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         92.173    1.402 65.752 0.000 89.426 94.921
RIAGENDRx[T.Male]  3.650    0.452  8.084 0.000  2.765  4.535
C(RIDRETH1)[T.2]   0.153    0.887  0.172 0.863 -1.586  1.891
C(RIDRETH1)[T.3]  -2.238    0.758 -2.954 0.003 -3.723 -0.753
C(RIDRETH1)[T.4]   3.098    0.836  3.707 0.000  1.460  4.737
C(RIDRETH1)[T.5]  -0.439    0.878 -0.500 0.617 -2.161  1.282
RIDAGEYR           0.474    0.013 36.482 0.000  0.449  0.500
BMXBMI             0.280    0.033  8.404 0.000  0.215  0.346
group Var          3.615    0.085                           
============================================================

"""

In [82]:
# Look at group Var: Variance of groups (clusters retreived from masking variables)
# If we choose two groups at random, their random effects differ on average
# by sqrt(2*'group Var') = 2.68
# This is equivalent to: Female -> Male; 6 years of aging
np.sqrt(2*3.615)

2.6888659319497505

#### 3.1.1 Predicted Random Effects

In [83]:
# Uniqueness of each county/cluster/group relative to the mean; difference wrt the mean;
# aka BLUPs
# Notice most extreme values differences:
# - cluster 1241, blood pressure 2.87
# - cluster 1282, blood pressure -3.58
result.random_effects

{1191: group   -1.630976
 dtype: float64,
 1192: group   -0.086162
 dtype: float64,
 1201: group   -2.042661
 dtype: float64,
 1202: group   -0.147472
 dtype: float64,
 1211: group    0.280623
 dtype: float64,
 1212: group    1.580732
 dtype: float64,
 1221: group    0.283347
 dtype: float64,
 1222: group    0.131512
 dtype: float64,
 1231: group   -2.038171
 dtype: float64,
 1232: group    0.617651
 dtype: float64,
 1241: group    2.878488
 dtype: float64,
 1242: group   -0.519364
 dtype: float64,
 1251: group    2.064967
 dtype: float64,
 1252: group    1.521281
 dtype: float64,
 1261: group   -1.261975
 dtype: float64,
 1262: group    0.980846
 dtype: float64,
 1271: group    0.118031
 dtype: float64,
 1272: group   -0.128397
 dtype: float64,
 1281: group   -0.384862
 dtype: float64,
 1282: group   -3.582111
 dtype: float64,
 1291: group   -3.271017
 dtype: float64,
 1292: group   -0.829538
 dtype: float64,
 1301: group   -0.884171
 dtype: float64,
 1302: group    2.790657
 dtype: f

### 3.2 Random Slopes

In [84]:
# Always center the time variable used to compute the slope
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())

In [86]:
# Now, the random effects have no intercept, but they have a slope, i.e.:
# 0+age_cen
# Note that here the level 2 formula is passed as a dictionary
model = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           vc_formula={"age_cen": "0+age_cen"},
           data=da)
result = model.fit()
result.summary()

/Users/mxagar/opt/anaconda3/envs/ds/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
===============================================================
Model:              MixedLM   Dependent Variable:   BPXSY1     
No. Observations:   5102      Method:               REML       
No. Groups:         30        Scale:                263.7323   
Min. group size:    106       Log-Likelihood:       -21469.9240
Max. group size:    226       Converged:            Yes        
Mean group size:    170.1                                      
---------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------
Intercept         115.207    1.209 95.265 0.000 112.836 117.577
RIAGENDRx[T.Male]   3.643    0.457  7.962 0.000   2.746   4.539
C(RIDRETH1)[T.2]    1.167    0.827  1.412 0.158  -0.453   2.787
C(RIDRETH1)[T.3]   -1.659    0.679 -2.444 0.015  -2.989  -0.328
C(RIDRETH1)[T.4]    3.610    0.739  4.884 0.000   2.161   5.058
C(RIDRETH1)[T.5]   -1.208    0.816 -1.480 0.139  -2.807   0.392
age_cen             0.467    0.018 26.235 0.000   0.432   0.502
BMXBMI              0.288    0.034  8.574 0.000   0.222   0.353
age_cen Var         0.004    0.000                             
===============================================================

"""

### 3.3 Intercepts and Slopes Allowed to Be Correlated

In [87]:
model = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group", re_formula="1+age_cen", data=da)
result = model.fit()
result.summary()

/Users/mxagar/opt/anaconda3/envs/ds/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
=================================================================
Model:                MixedLM   Dependent Variable:   BPXSY1     
No. Observations:     5102      Method:               REML       
No. Groups:           30        Scale:                255.4451   
Min. group size:      106       Log-Likelihood:       -21413.6193
Max. group size:      226       Converged:            Yes        
Mean group size:      170.1                                      
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           115.467    1.340 86.173 0.000 112.840 118.093
RIAGENDRx[T.Male]     3.662    0.451  8.121 0.000   2.778   4.546
C(RIDRETH1)[T.2]      0.023    0.898  0.025 0.980  -1.738   1.783
C(RIDRETH1)[T.3]     -2.251    0.778 -2.893 0.004  -3.775  -0.726
C(RIDRETH1)[T.4]      3.011    0.854  3.524 0.000   1.336   4.686
C(RIDRETH1)[T.5]     -0.585    0.893 -0.655 0.512  -2.336   1.165
age_cen               0.466    0.018 26.286 0.000   0.431   0.501
BMXBMI                0.283    0.033  8.497 0.000   0.218   0.349
group Var             8.655    0.169                             
group x age_cen Cov   0.119    0.004                             
age_cen Var           0.004    0.000                             
=================================================================

"""

## 4. Week 3 Assessment

In [90]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import statsmodels.api as sm
import pandas as pd  

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

da["smq"] = da.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

np.random.seed(123)

#### Q1: What is clustered data?

Data grouped according to some common property, such as geographical closeness, subject id, etc. Data within each cluster is often correlated, i.e., we have ependent data.

Data is considered clustered when observations are correlated within groups, sometimes related to study designs.

#### Q2: Which of the listed features has the highest correlation between two observations in the same cluster?

SDMVSTRA; that is to be expected, since SDMVSTRA is an identifier variable used to define clusters or groups.

In [91]:
for v in ["BPXSY1", "SDMVSTRA", "RIDAGEYR", "BMXBMI", "smq"]:
    model = sm.GEE.from_formula(v + " ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
SDMVSTRA The correlation between two observations in the same cluster is 0.959
RIDAGEYR The correlation between two observations in the same cluster is 0.035
BMXBMI The correlation between two observations in the same cluster is 0.039
smq The correlation between two observations in the same cluster is 0.026
